In [41]:
import torch
import numpy as np


In [42]:
print(torch.__version__)

2.1.2


In [43]:
t1=torch.tensor(4.0)
t1

tensor(4.)

In [44]:
t1.dtype

torch.float32

In [45]:
t2=torch.tensor([1.,2,3,4])
t2

tensor([1., 2., 3., 4.])

In [46]:
t2.dtype

torch.float32

In [47]:
t3=torch.tensor([[5.,6],
                 [7,8],
                 [9,10]])
t3

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])

In [48]:
t4=torch.tensor([
    [[11,12,13],
     [13,14,15]],
     [[15,16,17],
      [17,18,19]]
])
t4

tensor([[[11, 12, 13],
         [13, 14, 15]],

        [[15, 16, 17],
         [17, 18, 19]]])

In [49]:
print(t1)
t1.shape

tensor(4.)


torch.Size([])

In [50]:
print(t2)
t2.shape

tensor([1., 2., 3., 4.])


torch.Size([4])

In [51]:
print(t3)
t3.shape

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])


torch.Size([3, 2])

In [52]:
print(t4)
t4.shape

tensor([[[11, 12, 13],
         [13, 14, 15]],

        [[15, 16, 17],
         [17, 18, 19]]])


torch.Size([2, 2, 3])

In [53]:
# bnasic tensor operations
x=torch.tensor(3.0)
w=torch.tensor(4.,requires_grad=True)
b=torch.tensor(5.,requires_grad=True)
x,w,b

(tensor(3.), tensor(4., requires_grad=True), tensor(5., requires_grad=True))

In [54]:
x.is_leaf

True

In [55]:
y=w*x+b
y

tensor(17., grad_fn=<AddBackward0>)

In [56]:
# y.retain_grad()
y.backward()
print('dy/dx:',x.grad)
print('dy/dw:',w.grad)
print('dy/db:',b.grad)
print('dy/dw 2:',w.grad_fn)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)
dy/dw 2: None


In [57]:
w2=torch.tensor(42.,requires_grad=True)
b2=torch.tensor(52.,requires_grad=True)
y2=w2+b2

In [58]:
y2.backward
print(w2.grad)

None


In [59]:
t6=torch.full((3,2),42)
t6

tensor([[42, 42],
        [42, 42],
        [42, 42]])

In [60]:
t7=torch.concatenate((t3,t6))
t7

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.],
        [42., 42.],
        [42., 42.],
        [42., 42.]])

In [61]:
t8=torch.sin(t7)
t8

tensor([[-0.9589, -0.2794],
        [ 0.6570,  0.9894],
        [ 0.4121, -0.5440],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165]])

In [62]:
t9=t8.reshape(3,2,2)
t9

tensor([[[-0.9589, -0.2794],
         [ 0.6570,  0.9894]],

        [[ 0.4121, -0.5440],
         [-0.9165, -0.9165]],

        [[-0.9165, -0.9165],
         [-0.9165, -0.9165]]])

In [63]:
x=np.array([[1,2],
            [3,4.]])
x

array([[1., 2.],
       [3., 4.]])

In [64]:
y=torch.from_numpy(x)
y

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)

In [65]:
x.dtype,y.dtype

(dtype('float64'), torch.float64)

In [66]:
z=y.numpy()
z

array([[1., 2.],
       [3., 4.]])

Fundamentals of Pytorch and Numpy done.
Next progress towards linear regression znd gradient descent

Video course Timestamp 25.28 / 1.43.32  

In [67]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [68]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [69]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [70]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.2076, -0.8897, -0.6101],
        [ 0.3858,  0.1496,  0.4119]], requires_grad=True)
tensor([-0.5730,  1.0081], requires_grad=True)


In [71]:
inputs @ w.t()+b

tensor([[-101.5732,   56.9027],
        [-136.8060,   75.6371],
        [-173.2428,   78.5025],
        [ -82.5795,   62.0307],
        [-143.0171,   70.8168]], grad_fn=<AddBackward0>)

In [72]:
def model(x):
    return x @ w.t()+b

In [73]:
preds=model(inputs)
print(preds)

tensor([[-101.5732,   56.9027],
        [-136.8060,   75.6371],
        [-173.2428,   78.5025],
        [ -82.5795,   62.0307],
        [-143.0171,   70.8168]], grad_fn=<AddBackward0>)


In [74]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [75]:
diff=preds-targets
torch.sum(diff * diff)/diff.numel()

tensor(23586.8867, grad_fn=<DivBackward0>)

to get rid of negatives in diff, square numbers; * does a element wise multiplication

In [76]:
# mean squared error loss 
def mse(t1,t2):
    diff=t1-t2
    return torch.sum(diff*diff)/diff.numel()

In [77]:
loss=mse(preds,targets)
print(loss)

tensor(23586.8867, grad_fn=<DivBackward0>)


In [78]:
loss.backward()

In [79]:
print(w)
print(w.grad)

tensor([[-0.2076, -0.8897, -0.6101],
        [ 0.3858,  0.1496,  0.4119]], requires_grad=True)
tensor([[-16878.1211, -19399.8867, -11751.1904],
        [ -1755.3833,  -2792.2773,  -1558.7906]])


In [80]:
print(b)
print(b.grad)

tensor([-0.5730,  1.0081], requires_grad=True)
tensor([-203.6437,  -23.2220])


In [81]:
with torch.no_grad():
    w-=w.grad*1e-5
    b-=b.grad*1e-5

In [82]:
w,b

(tensor([[-0.0388, -0.6957, -0.4926],
         [ 0.4034,  0.1775,  0.4274]], requires_grad=True),
 tensor([-0.5710,  1.0083], requires_grad=True))

In [83]:
preds=model(inputs)
loss=mse(preds,targets)
print(loss)

tensor(16186.9375, grad_fn=<DivBackward0>)


the grad values for w and b should be reset to zero, else the next operation will add to existing grad values

In [84]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


to train the model repeat the above operation x number of times, called epochs

In [85]:
for i in range(100):
    preds=model(inputs)
    loss=mse(preds,targets)
    loss.backward()
    with torch.no_grad():
        w-=w.grad*1e-5
        b-=b.grad*1e-5
        w.grad.zero_()
        b.grad.zero_()

In [86]:
preds=model(inputs)
loss=mse(preds,targets)
print(loss)

tensor(271.0529, grad_fn=<DivBackward0>)


In [87]:
preds

tensor([[ 63.2267,  74.8270],
        [ 82.9522, 101.4358],
        [107.2514, 123.9571],
        [ 56.5236,  62.0644],
        [ 82.4230, 106.0507]], grad_fn=<AddBackward0>)

In [88]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])